<a href="https://www.kaggle.com/code/andrey36912/notebook6f2f962ec9?scriptVersionId=192999830" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import torch
from torchvision import datasets, transforms
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data_dir = '/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0'


dataset = datasets.ImageFolder(root=data_dir)

In [27]:
data_dir = '/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1'
dataset1 = datasets.ImageFolder(root=data_dir)

In [ ]:
data_dir = '/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_2'
dataset2 = datasets.ImageFolder(root=data_dir)

In [ ]:
from torch.utils.data import ConcatDataset
dataset = ConcatDataset([dataset, dataset1, dataset2])
len(dataset)

In [ ]:
classes = dataset.datasets[0].classes
num_classes = len(dataset.datasets[0].classes)
model = TransferLearningClassifier(num_classes).to(device)

In [ ]:
from torch.utils.data import random_split

train_ratio = 0.8

train_data, test_data = random_split(dataset, [train_ratio, 1-train_ratio])

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.3517, 0.3557, 0.3570],
                         std=[0.2325, 0.2347, 0.2353]) 
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.3517, 0.3557, 0.3570],
                         std=[0.2325, 0.2347, 0.2353]) 
])


class TransformDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y
        
    def __len__(self):
        return len(self.subset)

    
train_data = TransformDataset(train_data, transform = train_transform)
test_data = TransformDataset(test_data, transform = test_transform)

In [ ]:
means = []
stds = []
for img, _ in train_data:
    means.append(torch.mean(img, [1, 2]).tolist())
    stds.append(torch.std(img, [1, 2]).tolist())

mean = torch.mean(torch.tensor(means), [0])
std = torch.mean(torch.tensor(stds), [0])

mean, std

In [ ]:
img.shape

In [ ]:
batch_size = 256
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
from torchvision import models

vgg19 = models.vgg19_bn(pretrained=True)
vgg19

In [ ]:
from torch import nn
import torch.nn.functional as F
import numpy as np


class TransferLearningClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        vgg = models.vgg19_bn(pretrained=True)
        
        # від'єднання градієнтів
        for param in vgg.parameters():
            param.requires_grad = False
        
        # кількість нейронів на виході
        in_features = vgg.classifier[0].in_features
        
        # деактивація останнього шару
        vgg.classifier = nn.Identity()
        
        # створення потрібних шарів
        self.feature_extractor = vgg
        
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(in_features, num_classes)
        

    def forward(self, x):
        out = self.feature_extractor(x) # (batch, in_features)
        
        out = self.dropout(out)
        out = self.linear(out)
        
        return out


    def predict(self, X, device='cpu'):
        X = torch.FloatTensor(np.array(X)).to(device)

        with torch.no_grad():
            y_pred = F.softmax(self.forward(X), dim=-1)

        return y_pred.cpu().numpy()


model = TransferLearningClassifier(len(dataset.classes)).to(device)

In [ ]:
!pip install -q torchsummary

In [28]:
from torchsummary import summary

summary(model, input_size=(3, 224, 224))

ModuleNotFoundError: No module named 'torchsummary'